In [196]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

In [197]:
weather_data = (r"OneDrive\Desktop\DataViz\DU-DEN-DATA-PT-11-2019-U-C\02-Homework\06-Python-APIs\Instructions\starter_code\weather_data.csv")
weather_df = pd.read_csv('weather_data.csv')
weather_df.head()

,Unnamed: 0,City,Lat,Lng,Humidity,Cloudiness,Wind Speed,Country,Max Temp,Date
0,0,Ribeira Grande,38.52,-28.70,82,20,14.99,PT,64.40,1580061680
1,1,Hilo,19.73,-155.09,64,40,8.05,US,68.00,1580061606
2,2,Punta Arenas,-53.15,-70.92,54,20,31.09,CL,57.20,1580061600
3,3,Portland,45.52,-122.68,81,40,11.41,US,51.01,1580061426
4,4,Kapaa,22.08,-159.32,88,40,6.93,US,73.40,1580061431


In [199]:
# Configure gmaps with API key
gmaps.configure(g_key)


# Store 'Lat' and 'Lng' into  locations 
locations = weather_df[["Lat", "Lng"]].astype(float)

# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values
weather_df= weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)


# Create a Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations=locations,weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [200]:
weather_df.head()

,Unnamed: 0,City,Lat,Lng,Humidity,Cloudiness,Wind Speed,Country,Max Temp,Date
0,0,Ribeira Grande,38.52,-28.70,82,20,14.99,PT,64.40,1580061680
1,1,Hilo,19.73,-155.09,64,40,8.05,US,68.00,1580061606
2,2,Punta Arenas,-53.15,-70.92,54,20,31.09,CL,57.20,1580061600
3,3,Portland,45.52,-122.68,81,40,11.41,US,51.01,1580061426
4,4,Kapaa,22.08,-159.32,88,40,6.93,US,73.40,1580061431


In [201]:
#A max temperature lower than 80 degrees but higher than 70
#Wind speed less than 10 mph
#Zero cloudiness

df = weather_df.loc[weather_df["Cloudiness"] == 0]
df

df2 = df.loc[df["Max Temp"] > 70]
df2

df3 = df2.loc[df2["Max Temp"] < 80]
df3

df4 = df3.loc[df3["Wind Speed"] < 10]
df4

ideal_cond = pd.concat([df, df4], ignore_index=True, sort=False)
ideal_cond.dropna()
ideal_cond = pd.DataFrame(ideal_cond)
ideal_cond

,Unnamed: 0,City,Lat,Lng,Humidity,Cloudiness,Wind Speed,Country,Max Temp,Date
0,6,Bulls,-40.17,175.38,82,0,1.99,NZ,66.00,1580061780
1,21,Busselton,-33.65,115.33,37,0,9.40,AU,72.00,1580061638
2,26,Longyearbyen,78.22,15.64,84,0,2.24,SJ,-2.20,1580061678
3,35,Port Hedland,-20.32,118.57,89,0,8.05,AU,86.00,1580061643
4,44,Saldanha,-33.01,17.94,40,0,14.99,ZA,68.00,1580061660
...,...,...,...,...,...,...,...,...,...,...
69,177,Malakal,9.53,31.66,19,0,6.20,SS,76.68,1580061807
70,269,Yulara,-25.24,130.99,19,0,4.70,AU,75.20,1580061657
71,387,Celestun,20.87,-90.40,74,0,3.60,MX,76.28,1580061840
72,488,Suriapet,17.15,79.62,51,0,2.37,IN,74.14,1580061792


In [211]:
hotel_df = ideal_cond



# geocoordinates

target_search = "Hotel"
target_radius = 5000

# set up parameters dictionary
params = {
    "radius" : target_radius,
    "target_type" : target_search,
    'key': g_key,
    }

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    long = row["Lng"]
    params['location']= f'{lat},{long}'
    
    response = requests.get(base_url, params = params)
    hotel_data = response.json()
   
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_df['results'][0]['name']
    except (KeyError, IndexError):
        print("Missing data for this location. Skipping to next city")
   
    print(json.dumps(hotel_data, indent = 4))
    
# extract results
    results = hotel_data['results']
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'name'] = results[0]['name']
        hotel_df.loc[index, 'City'] = results[0]['City']
        hotel_df.loc[index, 'Country'] = results[0]['Country']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


Missing data for this location. Skipping to next city
{
    "html_attributions": [],
    "next_page_token": "CqQCFQEAALXI7KKMVa5-p2i-HYbr8_dm6gY41afXKfsIADyg7XUjPzXyALMMOxriehgqJwfNpK_ewX7l9shPe0c62_3D2wYMQQdZHDbKYt0B5FV-hqJK9cgZJmUTGI3b16OBOWc42svIf33KF28MI6w3-FTgbyNKSPZiF0u4Oot-7Mt_9THpoYJMJa5zbiYCEQgxQNEExiMbCUZfF32tAkldrEfOPQUknXckP3r1hoKv9QWiCYo_UcbrBhgtHGSkDp-Jfwd4Dbl6yBDnK3PoLc--iIRET9DmUtizQCddvgbcmwYRcSGKNOkayVgJWxLGYLN83E8wtpqWCYOwwf01ZvpEGCH3fgWcACplfYLvSpbriNciSp5OpS6L-OoGZyH1R-Lg8qCJ_hIQCIrrsJmTYRgzz2ukgbI9pRoUlleEQo_QV0HWCaVxHblejzJY4ds",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -40.06887510000001,
                    "lng": 175.3776327
                },
                "viewport": {
                    "northeast": {
                        "lat": -39.907328,
                        "lng": 175.5143616
                    },
                    "southwest": {
                        "lat": -40.140217,
 

In [212]:
hotel_df
hotel_df

,Unnamed: 0,City,Lat,Lng,Humidity,Cloudiness,Wind Speed,Country,Max Temp,Date,name
0,6,Bulls,-40.17,175.38,82,0,1.99,NZ,66.00,1580061780,Marton
1,21,Busselton,-33.65,115.33,37,0,9.40,AU,72.00,1580061638,Busselton
2,26,Longyearbyen,78.22,15.64,84,0,2.24,SJ,-2.20,1580061678,Longyearbyen
3,35,Port Hedland,-20.32,118.57,89,0,8.05,AU,86.00,1580061643,Port Hedland
4,44,Saldanha,-33.01,17.94,40,0,14.99,ZA,68.00,1580061660,Saldanha
...,...,...,...,...,...,...,...,...,...,...,...
69,177,Malakal,9.53,31.66,19,0,6.20,SS,76.68,1580061807,Malakal
70,269,Yulara,-25.24,130.99,19,0,4.70,AU,75.20,1580061657,Petermann
71,387,Celestun,20.87,-90.40,74,0,3.60,MX,76.28,1580061840,Celestún
72,488,Suriapet,17.15,79.62,51,0,2.37,IN,74.14,1580061792,Suryapet


In [213]:
# # Configure gmaps with API key
# gmaps.configure(g_key)


# # Store 'Lat' and 'Lng' into  locations 
# locations = weather_df[["Lat", "Lng"]].astype(float)

# # Convert Poverty Rate to float and store
# # HINT: be sure to handle NaN values
# weather_df= weather_df.dropna()
# humidity = weather_df["Humidity"].astype(float)


# # Create a Heatmap layer
# fig = gmaps.figure()

# heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
#                                  dissipating=False, max_intensity=100,
#                                  point_radius = 1)

# fig.add_layer(heat_layer)

# fig

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))